In [ ]:
import os
import sys


# this assumes that trinity-packet was installed in home directory
os.chdir(os.path.expanduser('~/trinity-packet/SAFE-NID'))
sys.path.append(os.path.expanduser('~/trinity-packet/code/'))

In [ ]:
from netflow.context_pcap.experiment import ContextPCAPExperiment

In [ ]:
# the configuration files include information on the ID and OOD samples 
# as well as the type of model used and other various parameters
# check the README for more information
config_filename = 'configs/experiments/OOD-Infiltration-model-ContextFNNBatchNormDropout-header_context.exp'
# what split to run the experiment on (generated from the train/test splits in data_processing.ipynb)
split_index = 0
# what gpu to run the experiment on
gpu = 1

In [ ]:
experiment = ContextPCAPExperiment(config_filename, split_index = split_index, gpu = gpu)

In [ ]:
# the model is not loaded or compiled by default
# need to compile the model before training, inference, or feature extraction can take place
experiment.compile_model()

In [ ]:
# train the model
experiment.run_training()

In [ ]:
# infer the model on both ID and OOD samples
# default is to run inference on the unbalanced data (heavily benign imbalanced)
experiment.run_inference()

In [ ]:
# extract the layers for this particular model for use in the model safeguard
# for each model, there is a instance variable extractable_layers
# this corresponds to the layers that we will use for the safeguard
layers = experiment.model.model.extractable_layers
for layer in layers:
    experiment.run_feature_extraction(layer)

In [ ]:
# use Gaussian kernel density model safeguard
for layer in experiment.model.model.extractable_layers:
    experiment.compute_robust_covariance_matrix(layer)
    experiment.compute_mahalanobis_distance(layer, balanced = False)

In [ ]:
# use normalizing flows model safe guard
for layer in experiment.model.model.extractable_layers:
    experiment.train_normalizing_flows(layer)
    experiment.calculate_normalizing_flows(layer, balanced = False)